In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# create element tree object 
tree = ET.parse('export.xml') 

In [3]:
# for every health record, extract the attributes into a dictionary (columns). Then create a list (rows).
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

In [5]:
# create DataFrame from a list (rows) of dictionaries (columns)
data = pd.DataFrame(record_list)
data.head()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
0,HKQuantityTypeIdentifierHeight,Health,11.0,cm,2017-10-07 17:18:21 +0300,2017-10-07 17:18:21 +0300,2017-10-07 17:18:21 +0300,167.64,NaN
1,HKQuantityTypeIdentifierHeight,Health,13.6,cm,2020-07-31 00:07:45 +0300,2020-07-31 00:07:45 +0300,2020-07-31 00:07:45 +0300,172,NaN
2,HKQuantityTypeIdentifierHeight,iPhone,14.0.1,cm,2020-09-30 02:16:13 +0300,2020-09-30 02:16:13 +0300,2020-09-30 02:16:13 +0300,173,NaN
3,HKQuantityTypeIdentifierBodyMass,Health,11.0,kg,2017-10-07 17:18:21 +0300,2017-10-07 17:18:21 +0300,2017-10-07 17:18:21 +0300,104,NaN
4,HKQuantityTypeIdentifierBodyMass,Health,13.6,kg,2020-07-31 00:07:45 +0300,2020-07-31 00:07:45 +0300,2020-07-31 00:07:45 +0300,105,NaN


In [6]:
data.columns

Index(['type', 'sourceName', 'sourceVersion', 'unit', 'creationDate',
       'startDate', 'endDate', 'value', 'device'],
      dtype='object')

In [7]:
data.unit.value_counts()

Cal          181126
count         55969
km            52801
count/min     50858
dBASPL        17078
min           14144
%              8391
km/hr          7078
cm             7063
ms             1693
m/s            1222
m                15
kg                5
mL/min·kg         3
hr                2
Name: unit, dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402292 entries, 0 to 402291
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   type           402292 non-null  object
 1   sourceName     402292 non-null  object
 2   sourceVersion  402292 non-null  object
 3   unit           397448 non-null  object
 4   creationDate   402292 non-null  object
 5   startDate      402292 non-null  object
 6   endDate        402292 non-null  object
 7   value          402284 non-null  object
 8   device         388037 non-null  object
dtypes: object(9)
memory usage: 27.6+ MB
